# Projeto Final de Ciência dos Dados. ( PkmnID)

## Algoritmo de predição da categoria de Pokémons por meio de suas imagens.
### O algoritmo realiza a extração e a clusterização de features de imagens por meio do método \"Bag of Visual Words\" (BOVW),classifica-as utilizando o método de machine learning \"Random Forest\" e prevê a categoria de Pokémons por meio de novas imagens.

In [ ]:
!pip install opencv-contrib-python
import cv2
import os
import os.path
import numpy as np
import math
import pandas as pd
import json
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
import matplotlib.pyplot as plt
from pprint import pprint
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

np.random.seed(0)

TRAIN_DIR = 'Assets/Final/Train_base'
SUPER_TRAIN_DIR = 'Assets/Final/Train_super'
TEST_DIR = 'Assets/Final/Test'
CLASSES = os.listdir(TRAIN_DIR)

NUM_CLUSTERS = 40

In [ ]:
## Máquina de controle:

# Re-adquiri as features das imagens.
READ_IMAGES = 0

# Cria arquivo .npy para dicionário de Features.
UPDATE_FILES = 0

# Constrói dataframe com dados do dicionário de features.
CREATE_FEATURE_DATAFRAME = 1

# Mostra em quais 'n' pokémons, cada feature é mais proeminente.
SHOW_TOP_N_FOR_FEATURES = 1

# Re-treina os models.
FIT_MODELS = 1

# Produz matrizes de confusão para todos os modelos.
PLOT_CONFUSION_MATRIXES = 1

# Produz uma lista com a métrica precision@n para todos os modelos.
PRECISION_AT_N = 1

# Roda o modelo combinado dos 3 modelos originais.
RUN_SUPER_MODEL = 1


## 1- Extração de features de imagens: Bag of Visual Words
### Uma vez que o dataset se trata de um conjunto de imagens de diferentes Pokémons, é necessário inicialmente extrair features dessas imagens, através do método "Bag of Visual Words".
### Com as imagens transformadas em features clusterizadas, elas são separadas em categorias de treino e teste, que serão utilizadas posteriormente pelo algoritmo de machine learning.
### O código abaixo realiza essas duas etapas:
#### Obs: Código produzido com a assistência do Prof. Fábio Ayres

In [ ]:
def get_img_names(TRAIN_DIR = TRAIN_DIR, SUPER_TRAIN_DIR = SUPER_TRAIN_DIR, TEST_DIR = TEST_DIR):

    TRAIN_IMG = []
    TEST_IMG = []
    TRAIN_LABEL = []
    TEST_LABEL = []
    SUPER_TRAIN_IMG = []
    SUPER_TRAIN_LABEL = []

    for train, sup_train, test in zip(os.listdir(TRAIN_DIR), os.listdir(SUPER_TRAIN_DIR), os.listdir(TEST_DIR)): 
        #Tecnicamente são iguais, mas não custa garantir.
        dir_train = os.listdir(os.path.join(TRAIN_DIR,train))
        dir_sup_train = os.listdir(os.path.join(SUPER_TRAIN_DIR,sup_train))
        dir_test = os.listdir(os.path.join(TEST_DIR,test))
        for img_train, sup_img_train, img_test in zip(dir_train, dir_sup_train, dir_test):
            TRAIN_IMG.append(os.path.join(TRAIN_DIR, train, img_train))
            TEST_IMG.append(os.path.join(TEST_DIR, test, img_test))
            TRAIN_LABEL.append(train)
            TEST_LABEL.append(test)
            SUPER_TRAIN_IMG.append(os.path.join(SUPER_TRAIN_DIR, sup_train, sup_img_train))
            SUPER_TRAIN_LABEL.append(sup_train)

    return TRAIN_IMG, TEST_IMG, SUPER_TRAIN_IMG, SUPER_TRAIN_LABEL, TRAIN_LABEL, TEST_LABEL

def cria_vocabulario(imagens, num_clusters):
    km = cv2.BOWKMeansTrainer(num_clusters)
    akaze = cv2.KAZE_create()
    for p in imagens:
        img = cv2.imread(p, cv2.IMREAD_GRAYSCALE)
        mask = np.ones(img.shape)
        kp, desc = akaze.detectAndCompute(img, mask)
        km.add(desc)
    return km.cluster()

def representa(vocab, img):
    kaze = cv2.KAZE_create()
    kp = kaze.detect(img)
    bowdesc = cv2.BOWImgDescriptorExtractor(kaze, cv2.FlannBasedMatcher())
    bowdesc.setVocabulary(vocab)
    return bowdesc.compute(img, kp)

def transforma_imagens(imagens, vocab):
    X = []
    for p in imagens:
        img = cv2.imread(p, cv2.IMREAD_GRAYSCALE)
        X.append(representa(vocab, img).flatten())
    return np.array(X)

def show_example(path = os.listdir("Testes/Testes/")[0], plot = True):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img_resized = cv2.resize(img, dsize=(120, 120))
    if Plot:
        plt.imshow(img_resized, cmap='gray', vmin=0, vmax=255)
    return representa(vocab, img_resized)

In [ ]:
Train_Dict = {}
Test_Dict = {}
Super_Train_Dict = {}
TRAIN_IMG, TEST_IMG, SUPER_TRAIN_IMG, SUPER_TRAIN_LABEL, TRAIN_LABEL, TEST_LABEL = get_img_names()

if READ_IMAGES:       
    vocab = cria_vocabulario(TRAIN_IMG, NUM_CLUSTERS)
    for pkmn in os.listdir(TRAIN_DIR):
        Train_Dict[pkmn] = transforma_imagens([os.path.join(TRAIN_DIR, pkmn, n) for n in os.listdir(os.path.join(TRAIN_DIR,pkmn))], vocab)   
    for pkmn in os.listdir(TEST_DIR):
        Test_Dict[pkmn] = transforma_imagens([os.path.join(TEST_DIR, pkmn, n) for n in os.listdir(os.path.join(TEST_DIR,pkmn))], vocab)
    for pkmn in os.listdir(SUPER_TRAIN_DIR):
        Super_Train_Dict[pkmn] = transforma_imagens([os.path.join(SUPER_TRAIN_DIR, pkmn, n) for n in os.listdir(os.path.join(SUPER_TRAIN_DIR,pkmn))], vocab)  
    
    if UPDATE_FILES:
        if 'files' not in os.listdir("Assets/"):
            os.mkdir('Assets/files')
        np.save('Assets/files/Features_Train', Train_Dict)
        np.save('Assets/files/Features_Test', Test_Dict)
        np.save('Assets/files/Features_Super_Train', Super_Train_Dict)
        np.save('Assets/files/Bag_of_Visual_Words', vocab)
            

else:
        Train_Dict = np.load('Assets/files/Features_Train.npy', allow_pickle=True)[()]
        Super_Train_Dict = np.load('Assets/files/Features_Super_Train.npy', allow_pickle=True)[()]
        Test_Dict = np.load('Assets/files/Features_Test.npy', allow_pickle=True)[()]
        vocab = np.load('Assets/files/Bag_of_Visual_Words.npy', allow_pickle=True)[()]

In [ ]:
X_train_vectors = np.array([v for k,v in Train_Dict.items()])
X_super_train_vectors = np.array([v for k,v in Super_Train_Dict.items()])
X_test_vectors = np.array([v for k,v in Test_Dict.items()]) 
y_train_vectors = [] 
y_super_train_vectors = []
y_test_vectors = []

for k in Train_Dict:
    x = []
    for v in Train_Dict[k]:
        x.append(k)
    x = np.array(x)
    y_train_vectors.append(x)

for k in Super_Train_Dict:
    x3 = []
    for v in Super_Train_Dict[k]:
        x3.append(k)
    x3 = np.array(x3)
    y_super_train_vectors.append(x3)

for k in Test_Dict:
    x2 = []
    for v in Test_Dict[k]:
        x2.append(k)
    x2 = np.array(x2)
    y_test_vectors.append(x2)

y_train_vectors = np.array(y_train_vectors)
y_super_train_vectors = np.array(y_super_train_vectors)
y_test_vectors = np.array(y_test_vectors)

X_train, X_super_train, X_test, y_train, y_super_train, y_test = [], [], [], [], [], []

for pkmn, matrix in zip(CLASSES, X_train_vectors):
    for feature_vector in matrix:
        X_train.append(feature_vector)
        y_train.append(pkmn)

for pkmn_super, matrix_super in zip(CLASSES, X_super_train_vectors):
    for feature_vector_super in matrix_super:
        X_super_train.append(feature_vector_super)
        y_super_train.append(pkmn_super)

for pkmn_test, matrix_test in zip(CLASSES, X_test_vectors):
    for feature_vector_test in matrix_test:
        X_test.append(feature_vector_test)
        y_test.append(pkmn_test)

X_train, X_super_train, X_test = np.array(X_train), np.array(X_super_train), np.array(X_test) 
y_train, y_super_train, y_test = np.array(y_train), np.array(y_super_train), np.array(y_test)

## 2 - Análise Exploratória:
### Para realizar a análise exploratória seguiremos alguns passos:

## 2.2 - Criar um dataframe para trabalhar melhor com o dataset:

In [ ]:
if CREATE_FEATURE_DATAFRAME:    
    lista = []
    for k in Train_Dict:
        mean = []
        for i in range(NUM_CLUSTERS):
            mean.append(pd.Series(Train_Dict[k][:,i]).mean())
        lista.append(mean)
    df_medias = pd.DataFrame(lista, index = CLASSES)
else:
    df_medias = pd.DataFrame([[1],[1]])

## Tabela das frequências relativas médias de cada feature por pokémon:

In [ ]:
df_medias

## 2.3 - Calculando os valores médios dos dados:
### Nesta etapa foi calculado os valores médios dos dados, e em sequência foram aproximados do ponto (0,0), origem do sistema.

In [ ]:
df_medias = df_medias - (1/NUM_CLUSTERS)

In [ ]:
df_medias.sum(axis=1)

In [ ]:
normas = (df_medias*df_medias).sum(axis=1)
for m in normas.index:
    df_medias.loc[m] = df_medias.loc[m]/np.sqrt(normas[m])

## 2.4 - Comparação entre os Pokémons:
### Com base nos valores calculados anteriormente, foi criada a tabela seguinte, que mostra o quanto os Pokémons são semelhantes entre si, sendo 1 a semelhança máxima, e -1 o oposto.

In [ ]:
df_compara = df_medias.dot(df_medias.transpose())
df_compara

### Podemos observar que alguns Pokémons possuem muitas semelhanças pois apresentam as mesmas features em abundância (na média)."

In [ ]:
if SHOW_TOP_N_FOR_FEATURES:    
    monstros = []
    for feat in range(NUM_CLUSTERS):
        monstros.append(sorted(df_medias.nlargest(n=5, columns=[feat]).index) + [feat])
    x = sorted(monstros)
    pprint(x)

### A soma das colunas da tabela anterior mostra quais Pokémons são mais difíceis de distinguir.

In [ ]:
df_compara.sum(axis = 1).sort_values(ascending = False)

## 3 - "Machine Learning" e Classificação:
### O método de aprendizado de máquina e classificação utilizado foi o "Random Forest Classifier", assim como "Logistic Regression Classifier" e "KNearesNeighbors Classifier".

In [ ]:
if FIT_MODELS:
    # Random forest
    randf = RandomForestClassifier(n_jobs=-1, random_state=0, n_estimators = 100)
    randf.fit(X_train, y_train)

    # KNearestNeighbors
    neigh = KNeighborsClassifier(n_neighbors=5)
    neigh.fit(X_train, y_train)
    
    # Decision Tree
    tree = DecisionTreeClassifier(random_state=0)
    tree.fit(X_train, y_train)

    # Logistic regression, não utilizado.
    #logit = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr').fit(X_train, y_train)
    #logit.fit(X_train, y_train);

    # Método Nearest Centroid, não utilizado.
    # from sklearn.neighbors.nearest_centroid import NearestCentroid
    # clf4 = NearestCentroid()
    # clf4.fit(X_train, y_train)

    # Método Support Vector Machine, não utilizado.
    # from sklearn import svm
    # clf5 = svm.SVC(gamma='scale')
    # clf5.fit(X_train, y_train)

### Abaixo armazenamos os modelos numa estrutura que nos será mais acessível.

In [ ]:
if FIT_MODELS:
    models = {'Random Forest': randf,
            'KNearestNeighbors': neigh,
            'Decision Tree': tree}
else:
    models = {0:0}

## 3.1 - Análise das classificações realizadas pelo modelo:
### A matriz de confusão abaixo mostra em mais detalhes os erros e acertos do classificador. É possível identificar que na maioria das vezes que o modelo falhou, ele identificou erroneamente o Pokémon como sendo uma "Jigglypuff" ou um "Arcanine".

#### Obs: A função *plot_confusion_matrix* abaixo não é de nossa autoria, e sua versão original pode ser encontrada no seguinte endereço: https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=True,
                          title=None,
                          cmap=plt.cm.Greens):
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        #print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    #print(cm)

    fig, ax = plt.subplots(figsize = (16,16))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

def multi_confusion_mtx(X_test, y_test, model_dict):
    for k,v in model_dict.items():
        plot_confusion_matrix(y_test, v.predict(X_test), classes=v.classes_,
                            title='Normalized confusion matrix for model %s' % k)
        plt.show()

def precision_at_n(model, n = 3, X_test = X_test, y_test = y_test):
    prediction = model.predict_proba(X_test)
    prediction_at_n = []
    for k in prediction:
        prediction_at_n.append(pd.Series(k, index = CLASSES).nlargest(n).index.tolist())
    hits, misses = 0, 0
    for guess, answer in zip(prediction_at_n, y_test):
        if answer in guess:
            hits += 1
        else:
            misses += 1
    return hits/(hits+misses)

def show_guess(path, model):
    return pd.Series(model.predict_proba(show_example(path))[0], index = model.classes_).sort_values(ascending = False)

np.set_printoptions(precision=2)

In [ ]:
if PLOT_CONFUSION_MATRIXES and FIT_MODELS:
    multi_confusion_mtx(X_test, y_test, models)

In [ ]:
if PRECISION_AT_N and FIT_MODELS:
    pprint({model_name: precision_at_n(model) for (model_name, model) in models.items()})

In [ ]:
for model_name, model in models.items():
    print("%s score of: %.5f" % (model_name, model.score(X_test, y_test)))

## Nada mal! Mas podemos fazer melhor?

## Stacking: Combinando os modelos em um(a) supermodelo.

In [ ]:
def predict_lower_models(X, models = models):    
    super_matrix = {}
    for model_name, model in models.items():
        super_matrix[model_name] = model.predict_proba(X).tolist()
    super_X_train_comb = []
    for vector_1, vector_2, vector_3 in zip(super_matrix['Random Forest'], super_matrix['KNearestNeighbors'], super_matrix['Decision Tree']):    
        super_X_train_comb.append(np.concatenate((vector_1, vector_2, vector_3), axis=None))
    return np.array(super_X_train_comb)

In [ ]:
super_X_train_comb = predict_lower_models(X_super_train)

In [ ]:
super_clf = RandomForestClassifier(n_jobs=-1, random_state= 0, n_estimators = 200)
super_clf.fit(super_X_train_comb, y_super_train)
super_clf.score(predict_lower_models(X_test), y_test)  

### Precison at N do Super Modelo

In [ ]:
precision_at_n(super_clf, X_test = predict_lower_models(X_test))

### Matriz de confusão do Super Modelo

In [ ]:
plot_confusion_matrix(y_test, super_clf.predict(predict_lower_models(X_test)), classes = CLASSES, title='Normalized confusion matrix for Super Model', cmap = plt.cm.Greens)
plt.show()